In [1]:
import requests
import re
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
api_key = os.getenv("lechat_api_key")

In [4]:
# api_key = "TqwpZsnTmT1WQG5A6T3fzDomj4PwZ0hn" # Replace with your actual API key

In [5]:
def ask_le_chat(api_key, question, temperature=0.7):
    api_endpoint = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-tiny", # or another engine if available
        "messages": [{"role": "user", "content": question}],
        "temperature": temperature
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Assuming the response contains the answer in a JSON format
        answer = response.json()
        return answer.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"


In [6]:
# 1. 확률 답변 요청

question = """
2025년 5월 1일부터 오늘까지의 한글/영문/중문문 기사 중 니켈 가격 변동을을 조사한 후,
답변은 니켈 가격이 상승할 확률과 니켈 가격이 하락할 확률 숫자만 보여주세요.
예시) 니켈 가격이 상승할 확률: 70%, 하락 확률: 30%
"""

In [7]:
answers = []
for i in range(10):
    answer = ask_le_chat(api_key, question, temperature=0.9)
    answers.append(answer)
    print(answer)


니켈 가격이 상승할 확률: 65%, 하락할 확률: 35% (이 정보는 현재의 시장 상황과 기간에 따라 변할 수 있으니 참고 용도로만 사용하시길 바랍니다.)
니켈 가격이 상승할 확률: 60%, 하락 확률: 40% (이 데이터는 가상 데이터입니다. 실제 데이터에 대한 예측은 부정확하며, 이용자가 유의해야 할 것으로 인지하시기 바랍니다.)
기사 조사는 현재 진행 중입니다. 결과를 약 30분 후 공개할 예정입니다.

 Niçl price change probability:
 UP: 60%
 DOWN: 40%
Nixture 가격이 상승할 확률: 60%, 하락 확률: 40%

(이 데이터는 하이퍼관례에 기반하여 생성되었습니다)
니켈 가격이 상승할 확률: 60%, 하락 확률: 40% (이 수치는 모델 예측 결과이며, 정확성은 보장되지 않습니다.)
정보를 구하는 중입니다. 결과 기다리jàman

여기에는 현재 정보를 찾을 수 없습니다. 따라서 가격 변동 확률은 정확히 알 수 없습니다. 따라서 평균만 기재하겠습니다.


답변:  Ни켈 가격이 상승할 확률: 65%, 하락 확률: 35% (이 답변은 예측이며 정확성은 보장되지 않습니다.)
데이터는 없어 확률을 친 nineteenbots! 그러나 2023년을 기준으로 하여 2025년 5월까지 니켈 가격 변동을 분석하여 아래와 같은 추세를 발견했습니다.

첨부한 그래프는 원틱 베이스로 이루어진 니켈 가격 데이터입니다. (원틱당 1kg)

2023년 초 시작한 것으로 보아 2023년 동안 니켈 가격이 크게 상승했습니다. 특히 2023년 6월부터 7월까지 가장 크게 올랐습니다.

그러나, 이후 2023년 8월부터 2023년 12월까지는 니켈 가격이 조금씩 하락했습니다.

2024년에는 에너지 비용을 포함한 세계 궈듭 문제와 함께 농업 업계에 다양한  challanges가 있었습니다. 그러나 그 결과로 2024년 3월까지는 니켈 가격이 기존보다 높게 유지되었습니다.

그러나 2024년 4월부터 5월까지 농업업계에 대한 특정한 개선사항으로

In [8]:
def extract_first_two_percent_values(text):
    """
    주어진 텍스트에서 앞에 등장하는 두 개의 '숫자%' 값을 찾아 숫자(int)로 반환합니다.
    
    Parameters:
        text (str): 분석할 텍스트
    
    Returns:
        tuple: (첫 번째 정수, 두 번째 정수), 두 개를 못 찾으면 None
    """
    # 정규표현식으로 '숫자%' 형태 추출
    matches = re.findall(r'(\d{2})%', text)
    
    if len(matches) >= 2:
        # 앞에서 두 개만 정수로 변환해서 반환
        return float(matches[0]), float(matches[1])
    else:
        return None

In [9]:
answers_float = []
for text in answers:
    if text is not None:
        temp = extract_first_two_percent_values(text)
        if temp is not None and temp[0] + temp[1] == 100:
            answers_float.append(temp)

In [10]:
answers_float

up, down = 0, 0
for u, d in answers_float:
    up += u
    down += d
up = up / len(answers_float)
down = down / len(answers_float)

answers_float.append((round(up, 1), round(down, 1)))
answers_float

[(65.0, 35.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (80.0, 20.0),
 (65.0, 35.0),
 (60.0, 40.0),
 (63.9, 36.1)]

In [11]:
pd.DataFrame(answers_float, columns=['상승 확률', '하락 확률'], 
             index=[str(i)+"회" for i in range(1, len(answers_float))] + ["평균"]).to_csv("환율_확률_결과.csv", encoding='utf-8')

In [12]:

question = """
2025년 5월 1일부터 오늘까지의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

| 요인 | 설명 |
| --- | --- |
| 1. 국내 경제 성장 | 국내 경제 성장은 원화의 가치를 높이는 요인 중 하나입니다. 국내 경제 성장은 국내 소비, 투자, 출口 등 원화 수요가 증가하여 원화의 가치를 높이는 역할을 합니다. |
| 2. 외국 경제 불안 | 외국 경제 불안은 원화의 가치를 높이는 요인 중 하나입니다. 외국 경제 불안은 외국 투자자들이 원화로 투자하기 보다 금융 상품 등 안전한 자산으로 투자하는 경향이 높아지며, 이로 인해 원화의 가치가 상승합니다. |
| 3. 통화 정책 | 통화 정책은 원화의 가치를 높이는 요인 중 하나입니다. 중앙은행이 통화 수익을 투자하거나 통화 공급을 제한하는 등 통화 공급 증가를 제어하는 정책으로 인해 원화의 가치가 상승할 수 있습니다. |
| 4. 국제 금융 시장 조건 | 국제 금융 시장 조건은 원화의 가치를 높이는 요인 중 하나입니다. 국제 금융 시장의 상황에 따라 원화 수요가 증가하거나 감소할 수 있고, 이로 인해 원화의 가치가 상승하거나 하락할 수 있습니다. 예를 들어, 국제 금융 시장의 상황이 불안해지면 투자자들이 안전한 자산으로 투자하기 위해 원화로 투자하는 경향이 높아지며, 이로 인해 원화의 가치가 상승할 수 있습니다. |


In [13]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [14]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_강세_요인.csv", encoding='utf-8', index=False)

In [15]:
question = """
2025년 5월 1일부터 오늘까지의의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

다음은 2022년 1월 1일부터 2023년 1월 31일까지 발행된 한국어, 영어, 중국어 기사 중 원/달러 환율과 달러화 내용을 조사한 결과, 최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) 형식으로 보여드리겠습니다.

| 요인 | 설명 |
| --- | --- |
| 경제 불안정성 | 국내 경제 성장 속도의 느린 흐름, 외국 경제 불안정성, 외국 금융 시장의 혼란, 국내 정부 재정 불안정성 등이 원화의 가치를 하락시키는 요인입니다. |
| 외국 금융 정책 | 외국 중앙은행이 인플레이션 대응 및 경제 조절을 위해 인터벌 등을 적용하면서 원화 가치를 하락시킬 수 있습니다. 또한 외국 금융 정책의 변동에 따라 원화 가치가 변동할 수 있습니다. |
| 재정 불안정성 | 국내 정부 재정 불안정성, 국내 외부 채무 부담, 국내 재정 부족 등이 원화의 가치를 하락시키는 요인입니다. 국내 정부 재정 불안정성이 증가할 경우 외국 투자자들이 국내 재정 위험에 대한 우려를 가지고 원화 투자를 줄이고, 이로 인해 원화의 가치가 하락할 수 있습니다. |
| 국제 경제 상황 | 국제 경제 상황의 변화, 특히 외국 경제 불안정성, 외국 금융 시장의 혼란, 외국 정부 재정 불안정성 등이 원화의 가치를 하락시키는 요인입니다. 특히 외국 경제 불안정성이 증가할 경우 외국 투자자들이 국내 투자를 줄이고, 이로 인해 원화의 가치가 하락할 수 있습니다. |


In [16]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [17]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_약세_요인.csv", encoding='utf-8', index=False)